# Data Aggregation

In [1]:
import pandas as pd
import numpy as np

Frequently we need to apply an *aggregation* operation to a data frame, for example:
* mean of values over a column,
* maximum or minimum values on a row,
* mean or standard deviation of data in different categories.

All these operations take a set of values and return a single value.

_pandas_ dataframes have some highly-optimized aggregation methods, such as

* `count()`, `sum()`, `prod()`
* `mean()`, `median()`
* `std()`, `var()`
* `min()`, `max()`

Let's generate a random dataframe to test some of these functions.

In [2]:
df = pd.DataFrame(np.random.randint(0,10,size=(6,3)), columns=["A","B","C"])
df

,A,B,C
0,3,4,3
1,4,0,2
2,1,6,7
3,0,9,2
4,0,6,0
5,4,4,3


In [3]:
df.sum()

A    12
B    29
C    17
dtype: int64

In [4]:
df.sum(axis="columns")

0    10
1     6
2    14
3    11
4     6
5    11
dtype: int64

In [5]:
df.std() # standard deviation

A    1.897367
B    2.994439
C    2.316607
dtype: float64

In [6]:
df.quantile(q=0.25) # first quartile

A    0.25
B    4.00
C    2.00
Name: 0.25, dtype: float64

# Split-apply-combine: groupby()

We may to break the data by the levels of some categorical data, and aggregate the data within those levels only.

Example:
* Average heights of men and women
* Total sales by region
* Average rainfall by month
* Subject averages in each class

The _split-apply-combine_ approach:

<img src="img/split-apply-combine.png" width="50%">

In [7]:
df = pd.DataFrame({"x": ["a","a","b","b","c","c"], "y":[2,4,0,5,5,10]})
df

,x,y
0,a,2
1,a,4
2,b,0
3,b,5
4,c,5
5,c,10


In [8]:
df.groupby("x").mean()

,y
x,
a,3.0
b,2.5
c,7.5


Consider the _Wholesale customers data_.

In [9]:
data = pd.read_csv("data_ex8_3.csv")
data.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicacies
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185


Suppose we want to get the totals, broken by regions:

In [10]:
data.groupby("Region").sum()

,Channel,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicacies
Region,,,,,,,
1,95,854833,422454,570037,231026,204136,104327
2,66,464721,239144,433274,190132,173311,54506
3,297,2910539,1437858,1866941,643483,680913,390302


Or, let's get the mean amount per transaction, grouped by sale channels.

In [11]:
data.groupby("Channel").mean()

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicacies
Channel,,,,,,,
1,2.342342,14294.414414,3387.545045,3994.729730,3899.40991,775.684685,1495.639640
2,2.533898,8955.364407,11418.822034,16808.661017,1686.20339,7509.813559,1839.855932


The `groupby()` method does not return anything by itself. It only prepares the data for function application.

In [12]:
data.groupby("Region")

This "lazy" approach is efficient and flexible. For example, we can get median revenue of only "Fresh" and "Milk" columns, grouped by region.

In [13]:
data.groupby("Region")[["Fresh","Milk"]].median()

,Fresh,Milk
Region,,
1,7363.0,3748.0
2,8090.0,2374.0
3,9546.0,3898.0


We can group by more than one category. The result is a data frame with a *hierarchical index*.

In [14]:
data.groupby(["Region","Channel"]).sum()

Fresh    Milk  Grocery  Frozen  Detergents_Paper  Delicacies
Region Channel                                                                
1      1         761233  228342   237542  184512             56081       70632
       2          93600  194112   332495   46514            148055       33695
2      1         326215   64519   123074  160861             13516       30965
       2         138506  174625   310200   29271            159795       23541
3      1        2085912  459174   526214  520296            102605      230435
       2         824627  978684  1340727  123187            578308      159867

Get the maximum revenue in one transaction, from each region and sale channel.

In [15]:
data.groupby(["Region","Channel"]).max()

Fresh   Milk  Grocery  Frozen  Detergents_Paper  Delicacies
Region Channel                                                              
1      1         56083  23527    16966   18711              5828        6854
       2         20782  28326    39694    8321             19410        6372
2      1         32717  16784    13626   60869              1679        5609
       2         27082  25071    67298   11559             38102        3508
3      1        112151  43950    21042   36534              6907       47943
       2         44466  73498    92780    8132             40827       16523

# Pivot tables

A *pivot table* is a rearrangement of the data, with index set to values of groups.

In [16]:
df = pd.DataFrame({"x": ["a","a","b","b","c","c","c", "d"], "y":[2,4,0,5,5,10,6,8]})
df

,x,y
0,a,2
1,a,4
2,b,0
3,b,5
4,c,5
5,c,10
6,c,6
7,d,8


A pivot table displaying the count of each group:

In [17]:
df.pivot_table(index="x", aggfunc="count")

,y
x,
a,2
b,2
c,3
d,1


Averages of each group:

In [18]:
df.pivot_table(index="x", aggfunc="mean")

,y
x,
a,3.0
b,2.5
c,7.0
d,8.0


Back to the wholesale customers data. We can get a table of mean revenue for a specific item category, grouped by channel and region.

In [19]:
data.pivot_table("Delicacies", index="Region", columns="Channel", aggfunc="mean")

Channel,1,2
Region,,
1,1197.152542,1871.944444
2,1105.892857,1239.000000
3,1706.925926,1973.666667


Or, we can display them all.

In [20]:
data.pivot_table(index="Region", columns="Channel", aggfunc="mean")

Delicacies              Detergents_Paper                      Fresh  \
Channel            1            2                1            2             1   
Region                                                                          
1        1197.152542  1871.944444       950.525424  8225.277778  12902.254237   
2        1105.892857  1239.000000       482.714286  8410.263158  11650.535714   
3        1706.925926  1973.666667       760.037037  7139.604938  15451.200000   

                            Frozen                   Grocery                \
Channel             2            1            2            1             2   
Region                                                                       
1         5200.000000  3127.322034  2584.111111  4026.135593  18471.944444   
2         7289.789474  5745.035714  1540.578947  4395.500000  16326.315789   
3        10180.580247  3854.044444  1520.827160  3897.881481  16552.185185   

                Milk                
Channel            1             2  
Region                              
1        3870.203390  10784.000000  
2        2304.250000   9190.789474  
3        3401.288889  12082.518519

Suppose we want to see the *total revenue* by region and channel, not broken by item categories.

First, generate a new column for the total of each transaction.

In [21]:
data["Total"] = data.iloc[:,2:].sum(axis="columns")
data.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicacies,Total
0,2,3,12669,9656,7561,214,2674,1338,34112
1,2,3,7057,9810,9568,1762,3293,1776,33266
2,2,3,6353,8808,7684,2405,3516,7844,36610
3,1,3,13265,1196,4221,6404,507,1788,27381
4,2,3,22615,5410,7198,3915,1777,5185,46100


Now we can use this new column for aggregation.

In [22]:
data.pivot_table("Total",index="Region", columns="Channel", aggfunc="sum")

Channel,1,2
Region,,
1,1538342,848471
2,719150,835938
3,3924636,4005400
